# Student Loan Risk with Deep Learning

In [34]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [35]:
# Read the csv into a Pandas DataFrame
file_path = "student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [36]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [37]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [38]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y.head()


0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [39]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.copy()


# Review the features DataFrame
X = X.drop("credit_ranking", axis=1)


### Step 3: Split the features and target sets into training and testing datasets.


In [40]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [41]:
# Create a StandardScaler instance
scaler = StandardScaler()


# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)


# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [42]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[50])


# Review the number of features
number_input_features


11

In [43]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  8


# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 5


# Define the number of neurons in the output layer
number_output_neurons = 1


In [44]:
# Create the Sequential model instance
nn = Sequential()

# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation="sigmoid"))


c:\Users\kevin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [45]:
# Display the Sequential model summary
nn.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 8)              │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 5)              │            45 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 147 (588.00 B)

 Trainable params: 147 (588.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [46]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [47]:
# Fit the model using 50 epochs and the training data
model = nn.fit(X_train_scaled, y_train, epochs=50)


Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 859us/step - accuracy: 0.4952 - loss: 0.6880 
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6296 - loss: 0.6610
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step - accuracy: 0.7020 - loss: 0.6322
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 422us/step - accuracy: 0.6744 - loss: 0.6150
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7097 - loss: 0.5876
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 846us/step - accuracy: 0.7215 - loss: 0.5606
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7127 - loss: 0.5601
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7190 - loss: 0.5432
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 426us/step - accuracy: 0.7181 - loss: 0.5500
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 874us/step - accuracy: 0.7202 - loss: 0.5434
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 396us/step - accuracy: 0.7307 - loss: 0.5284
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 845us/step - accu

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [48]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test, y_test, verbose=2)


# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


13/13 - 0s - 6ms/step - accuracy: 0.4800 - loss: 3.2785
Loss: 3.2785000801086426, Accuracy: 0.47999998927116394


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [49]:
# Set the model's file path
file_path = Path("student_loans.keras")


# Export your model to a keras file
nn.save(file_path)


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [50]:
# Set the model's file path
file_path = Path("student_loans.keras")


# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [54]:
# Make predictions with the test data
y_pred = nn_imported.predict(X_test)


# Display a sample of the predictions
y_pred[:10]


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 985us/step


array([[8.1331155e-04],
       [2.7307301e-06],
       [3.8835740e-01],
       [3.2960650e-04],
       [4.9988627e-01],
       [5.2924919e-01],
       [5.7793951e-01],
       [7.1730360e-12],
       [3.9593768e-01],
       [4.3086624e-01]], dtype=float32)

In [52]:
# Save the predictions to a DataFrame and round the predictions to binary results
results = pd.DataFrame(y_test)


### Step 4: Display a classification report with the y test data and predictions

In [53]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, y_pred.round()))


              precision    recall  f1-score   support

           0       0.47      0.99      0.64       188
           1       0.75      0.03      0.05       212

    accuracy                           0.48       400
   macro avg       0.61      0.51      0.35       400
weighted avg       0.62      0.48      0.33       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

To build a recommendation system for student loan options, the following data would be relevant and appropriate:

- **Student Demographics**: Age, gender, location, and socioeconomic background helps in understanding the student's financial needs and eligibility for certain loan options.
- **Academic Information**: GPA, major, and academic performance is crucial to assess the student's potential for successful loan repayment based on their academic achievements.
- **Financial Information**: Income, credit score, existing debts, and financial aid received is essential to evaluate the student's financial stability and ability to repay the loan.
- **Loan History**: Previous loans taken, repayment history, and any defaults helps in assessing the student's creditworthiness and risk profile.
- **Loan Preferences**: Preferred loan amount, interest rate, repayment period, and type of loan (federal or private) helps in tailoring the recommendations to the student's specific needs and preferences.

- Collecting this data ensures that the recommendation system can provide personalized and accurate loan options that align with the student's financial situation and academic goals.

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

- The model would be using Content-based Filtering. This is because the data collected is specific to the individual student's characteristics, such as demographics, academic information, financial information, loan history, and loan preferences. Content-based Filtering works by recommending items that are similar to those that the user has shown interest in, based on the attributes of the items. In this case, the recommendation system would analyze the student's data and match it with loan options that have similar attributes, ensuring that the recommendations are personalized and relevant to the student's unique profile.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

- **Data Privacy and Security**: Handling personally identifiable information such as financial data, academic records, and personal demographics requires stringent data privacy and security measures. Ensuring compliance with regulations like GDPR and FERPA is crucial to protect student data from breaches and misuse. Failure to secure this data can lead to legal consequences and loss of trust among users.

- **Bias and Fairness**: Ensuring that the recommendation system does not inadvertently favor or discriminate against certain groups of students is essential. Bias in the data or the algorithm can lead to unfair loan recommendations, which can exacerbate existing inequalities. It is important to regularly audit the system for bias and implement measures to promote fairness and inclusivity in loan recommendations.